# SIR Predictive Analysis Project
### By: Mohammed Rehan
Contributions: Zayn Bhatti, Jeremiah Chulliyadan, Nandan Varma, Huzayfa Jasat

## Introduction
This project explores the application of epidemiological models used to analyze and predict the spread of measles in the United States for the next 20 years. The SIR model is used to preform a predictive analysis using historical data on U.S. measles cases.

Inspired by Mr. P Solver's video $[2]$ on the creating a SIR model in python, this project uses the same approach to solve systems of differential equations using `odeint`, a python package from `scipy.integrate`. Historical data is used to derive all perameters being based on a CSV file $[1]$ containing recorded cases, which serve as the foundations to determining the transmission rate of measles.

Using seperate functions, this program calculates the infection rate constant from histoical data, solves a system of differential equations, and creates a interactive user interface.

It is important to note that I AM NOT AN EPIDEMIOLOGIST and this project is not intended to simulate or predict real-world scenarios. Instead, it serves as experience in Python libraries like `Matplotlib`, `SciPy`, and `NumPy`. The results presented are purely theoretical and are not reflective of real world epidemiological trends

Note: This project was created by Mohammed Rehan Ali Shaik. Tested by contributors. All functions, concepts, or ideas used from other sources are credited on the references page and in-text citations, using their respective identification (more information on the reference pages)


## The SIR Model

<img src='SIR Image.PNG'>

### SIR Model Explanation
#### Refernces: $[2][3][4]$

#### The SIR (Susceptible, Infected, Recovered) model is the simplest epidemiological model used to study the spread of infectious diseases.
#### This model uses a few assumptions to hold true
1. Fixed population size: The U.S. Population stays constant over the measurement period
2. Homogeneous Mixing: the population is assumed to mix randomly and evenly
3. Immunity after recovery: The recovered population cannot become susceptible
4. Doesn't factor  in intervention: Assumes no external intervention like vaccinations

#### With these assumptions, the SIR model breaks the U.S. populations into 3 separate populations:
1. Susceptible (S): Individuals who can contract the disease, The susceptible population decreases as individuals become infected.
2. Infected (I): Individuals currently infected and capable of spreading the disease, The infected population increases when susceptible individuals are infected and decreases as they recover.
3. Recovered (R): Individuals who have recovered and are immune, The recovered population increases as infected individuals recover.

#### Assuming population can only move forward in the model, differential equations are used to describe how the populations change over time

- Susceptible population:
  $\frac{dS}{dt} = -\frac{\beta}{N} S I$


- Infected population: $\frac{dI}{dt} = \frac{\beta}{N} S I - \gamma I$

- Recovered population:
  $\frac{dR}{dt} = \gamma I$


#### This system of equations is solved by `ODEINT` from `SciPy.Integrate`

### Parameters and Variables:
- $\beta$: Infection/Transmission rate constant.
- $\gamma$: Recovery rate constant $\frac{1}{Recovery Period (days)}$
- $N = S + I + R$: Total population.

#### This model helps analyze disease dynamics and predict the spread of infections using 8 functions


## Imports, Libraries, and Initialization

In [2]:
import numpy as np  #For computations and handling arrays, lists, and dictionaries
import matplotlib.pyplot as plt  # For creating and displaying plots and graphs
from scipy.integrate import odeint  # For solving differential equations numerically
import csv  # For reading CSV files
import tkinter as tk  # For building the GUI
import matplotlib  # To configure Matplotlib's rendering backend
from matplotlib.widgets import Slider  # For adding interactive sliders to plots (optional)

matplotlib.use('TkAgg')  # Set the backend to TkAgg for Tkinter integration
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg  # Embed Matplotlib plots into Tkinter GUI


In [3]:
data = []  # Empty list to store infected cases over time
beta_list = []  # Empty list to store transmission rates over time
previous_cases = 0  # Variable to track the number of cases in the previous time step, initially set to 0
previous_population = 0  # Variable to track the population in the previous time step, initially set to 0
gamma = 1 / 14  # Recovery rate [5]
population = 3349900000  # Total population size (assumed to be a fixed value, here 3.35 billion)
times = np.arange(0, 20, 1)  # Array representing time steps from 0 to 20 (e.g., days), with a step of 1



## Function 1: Reading CSV Function

The `reading_data()` function reads a CSV file containing U.S. measles data, processes each row as a dictionary, and removes any entries with empty or whitespace-only values. It then appends the cleaned data (dictionaries) to the `data` list for further use. The function ensures that only relevant, non-empty data is stored, making it ready for analysis or processing.

The CSV File used in the project is available here: https://github.com/owid/owid-datasets/blob/master/datasets/US%20Measles%20Cases%20and%20Deaths%20(OWID%2C%202017)/US%20Measles%20Cases%20and%20Deaths%20(OWID%2C%202017).csv

In [4]:
def reading_data():
    with open('US Measles Cases and Deaths (OWID, 2017).csv', 'r') as f:
        reader = csv.DictReader(f)
        for line in reader:
            cleaned_line = {key: value for key, value in line.items() if value and value.strip()}
            data.append(cleaned_line)

## Function 2: Calculating Transmition Rate Constant ($\beta$)

The `beta_value()` function calculates the average transmission rate (β) based on changes in reported cases and population. It iterates through the data, computes the change in cases and population, and calculates β using the formula:

$$
\beta = \frac{\text{Current Cases} - \text{Previous Cases}}{\text{Previous Cases} \times (\text{Current Population} - \text{Previous Population})}
$$
The function averages the β values and returns the result.

Note: This is a simplified method for calculating β, which may not fully reflect all real-world factors but is adequate for the scope of this project.

In [5]:
def beta_value(prev_pop, prev_case):
    for line in data:
        current_case = line.get('Reported Cases', '0').strip()
        current_pop = line.get('Reported Population', '0').strip()

        current_case = int(current_case) if current_case else 0
        current_pop = int(current_pop) if current_pop else 0

        if current_case == 0 or current_pop == 0:
            continue

        delta_case = current_case - prev_case
        delta_pop = current_pop - prev_pop

        if prev_pop != 0 and prev_case != 0:
            temp_beta = delta_case / (prev_case * delta_pop)
            beta_list.append(temp_beta)

        prev_case = current_case
        prev_pop = current_pop

    avg_beta = sum(beta_list) / len(beta_list) if beta_list else 0
    print(f'Average Transmission Rate: {avg_beta}')
    return avg_beta

## Function 3: Defining Mathematical Model
#### References: Function taken from $[2][3]$

`dadt(a, t, b, g, n)` defines the system of ordinary differential equations (ODEs) for the classic SIR model. It calculates the rates of change for the susceptible, infected, and recovered equations in the population over time
#### Parameters:
- `a` : List Containing the Current Value of S, I, R
- `t` :Time Variable Required for Solving ODE’s
- `b` : Transmission rate (\beta)
- `g` : Recovery Rate (\gamma)
- `n` : Total Population Size (N)
#### Returns
`dadt` returns a list of the rates of change:
- $\frac{dS}{dt} = -\frac{\beta}{N} S I$
- $\frac{dI}{dt} = \frac{\beta}{N} S I - \gamma I$
- $\frac{dR}{dt} = \gamma I$

In [6]:
def dadt(a, t, b, g, n):
    s, i, r = a
    return [
        -b / n * s * i,
        b / n * s * i - g * i,
        g * i
    ]

## Function 4: Initialize the model
#### References: Function taken from $[2][3]$
This function checks if the list is not empty. If it's not, it returns the last element `(list[-1])`; if it is empty, it returns `None`. It retrieves the most recent data from the list.

In [7]:
def initial_data(d):
    if d: # Check if the data list is not empty
        return d[-1]  # Return the last element in the data list, intializing the model
    else:
        return None  # Return None if the data list is empty

## Function 5: Solve Ordinary Differential Equations
#### References: Function taken from $[2][3]$
This function solves a system of differential equations, likely modeling disease spread, using `odeint` from `scipy.integrate`. It sets initial conditions for `susceptible`, `infected`, and `recovered` populations, then integrates the equations over time, storing the results in `S`, `I`, and `R`.


In [8]:
def solve_ode():
    # Retrieve the most recent reported cases, defaulting to 0 if no data is found
    iO = int(initial_data(data).get('Reported Cases', '0').strip()) if initial_data(data) else 0
    # Calculate the susceptible population by subtracting infected cases from total population
    sO = population - iO
    # Set initial number of recovered individuals to 0 for simplicity
    rO = 0

    # Create an initial condition list: susceptible, infected, and recovered populations
    yo = [sO, iO, rO]

    # Solve the system of differential equations using odeint, passing required parameters
    sol = odeint(dadt, yo, times, args=(beta_value(previous_population, previous_cases), gamma, population))

    # Unpack the solution array into individual arrays for susceptible, infected, and recovered populations
    global S, I, R
    S, I, R = sol.T

## Function 6: Update Graph
This function updates the plot when the slider is moved by retrieving the `time` and adjusting the position of a vertical line on the plot. It finds the closest time index in the data, then updates the labels for infected and recovered populations with the corresponding values at that time. Finally, it redraws the plot to reflect the updated positions and label texts.

In [9]:
def update(val):
    # Get the current position of the slider (time value)
    x_position = slider.val
    # Move the vertical line to the new x-position on the plot
    vertical_line.set_xdata([x_position])

    # Find the closest index in the 'times' array to the slider's current value
    idx = (np.abs(times - x_position)).argmin()

    # Retrieve the infected and recovered populations at the selected time index
    y_infected = I[idx]
    y_recovered = R[idx]

    # Move and update the text for the infected population label at the current x-position
    intersection_itext.set_position((x_position, y_infected))
    intersection_itext.set_text(f'({x_position:.2f}, {y_infected:.2f})')

    # Move and update the text for the recovered population label at the current x-position
    intersection_rtext.set_position((x_position, y_recovered))
    intersection_rtext.set_text(f'({x_position:.2f}, {y_recovered:.2f})')

    # Redraw the figure to reflect the changes
    fig.canvas.draw_idle()

## Function 6: User Interface

This function sets up the user interface for plotting and interacting with a disease prediction model. It first calls `reading_data()` then uses `solve_ode()` for solving the system of ordinary differential equations. Then crease a plot displaying the infected and recovered cases over time. A slider is added to allow the user to select diffent time points. The `Tkinter` library was used to create a window and display the plot.

In [10]:
def user_interface():
    # Read data and solve the differential equations
    reading_data()
    solve_ode()

    # Initialize global variables for the figure, axes, slider, and text elements
    global fig, ax, slider, vertical_line, intersection_itext, intersection_rtext

    # Create a new figure and axes for the plot
    fig, ax = plt.subplots()
    # Adjust the space for the slider at the bottom of the window
    plt.subplots_adjust(bottom=0.25)

    # Plot the infected and recovered cases over time with respective colors and labels
    ax.plot(times, I, label='Infected', color='blue')
    ax.plot(times, R, label='Recovered', color='green')
    # Add a vertical dashed line at time = 0 to mark the starting point
    vertical_line = ax.axvline(x=0, color='red', linestyle='--')
    # Add text labels for the infected and recovered populations at the start (position 0, 0)
    intersection_itext = ax.text(0, 0, "", color='blue', fontsize=10)
    intersection_rtext = ax.text(0, 0, "", color='green', fontsize=10)

    # Set the labels for the x and y axes, and the title of the plot
    ax.set_xlabel('Time')
    ax.set_ylabel('Cases')
    ax.set_title('Prediction Model')
    # Display the legend for the plot
    ax.legend()
    # Enable grid lines on the plot for better visibility
    ax.grid(True)

    # Define the area for the slider below the plot and create the slider
    ax_slider = plt.axes([0.2, 0.1, 0.65, 0.03], facecolor='lightgoldenrodyellow')
    slider = Slider(ax_slider, 'Time', times.min(), times.max(), valinit=0, valstep=(times[1] - times[0]))
    # Attach the update function to the slider's value change event
    slider.on_changed(update)

    # Create a Tkinter window to display the plot
    root = tk.Tk()
    root.wm_title("SIR Model Plot")  # Set the window title
    # Embed the plot in the Tkinter window using FigureCanvasTkAgg
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()  # Draw the plot on the canvas
    # Pack the canvas widget into the Tkinter window
    canvas.get_tk_widget().pack()
    # Start the Tkinter main event loop to display the window
    tk.mainloop()


## Run Program

In [ ]:
user_interface()

Average Transmission Rate: 0


## Results
#### Comparrison Data taken from $[6]$
In 2017, the United States reported 120 cases of measles, significantly higher than the usual annual reports of 1 to 2 cases, largely due to an outbreak in a largely unvaccinated Somali community in Minnesota. $[6]$ According to the predictions made by the SIR (Susceptible-Infected-Recovered) model, the estimated number of infections for the same year was 170.11.

$$
\text{Accuracy} = \left( 1 - \frac{| \text{Predicted} - \text{Actual} |}{\text{Actual}} \right) \times 100
$$

Using the predicted value of 170.11 and the actual value of 120:

$$
\text{Accuracy} = \left( 1 - \frac{|170.11 - 120|}{120} \right) \times 100 = \left( 1 - \frac{50.11}{120} \right) \times 100 \approx 58.18\%
$$

The model's prediction is approximately 58.18% accurate, indicating that it overestimated the number of infections. This suggests the model may not fully capture the complexity of the outbreak and could benefit from further refinement and adjustments.


## Improvements

- Refinement of Parameters: Regularly update infection and recovery rates based on real-time data and regional factors, such as vaccination coverage and sociodemographic variables, to improve model accuracy.

- Data Structures and Efficiency: Use more advanced data structures like `Pandas DataFrames` for better handling of time-series data, optimize calculations with vectorization and parallel processing techniques, and store large datasets in `SQL databases` for efficient querying and management.

- Incorporate More Complex Models: Extend the SIR model to more sophisticated models like `SEIR` or `SIRS`, which include additional compartments, and use stochastic processes to account for variability in disease transmission.


## References
[1] "US Measles Cases and Deaths (OWID, 2017)," Our World in Data, Available: https://github.com/owid/owid-datasets/blob/master/datasets/US%20Measles%20Cases%20and%20Deaths%20(OWID%2C%202017)/US%20Measles%20Cases%20and%20Deaths%20(OWID%2C%202017).csv, Accessed: Jan. 8, 2025.

[2] Mr P Solver, "The SIR Disease Model in PYTHON" YouTube, Jan. 2, 2020. [Online]. Available: https://www.youtube.com/watch?v=zpYMiJd3pqg. [Accessed: Jan. 8, 2025].

[3] L. Polson, "Python Metaphysics Series: Vid2," GitHub, Available: https://github.com/lukepolson/youtube_channel/blob/main/Python%20Metaphysics%20Series/vid2.ipynb, Accessed: Jan. 8, 2025.

[4] https://www.youtube.com/watch?v=f1a8JYAixXU

[5] https://www.canada.ca/en/public-health/services/diseases/measles/health-professionals-measles.html

[6] https://en.wikipedia.org/wiki/Epidemiology_of_measles?utm_source=chatgpt.com

### Reference Usage

- Sections containing concepts or ideas from external sources have been given reference at the section title
- Lines copied and pasted from external sources have been given in-text citation at the end of the respective sentence